In [28]:
from glob import glob
import os   
import sys
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from tqdm import tqdm

In [35]:
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        

def apply_windowing(arr, window_center, window_width):
    """
    Apply windowing to a DICOM pixel array with normalization.
    """
    # Compute window bounds
    lower_bound = window_center - (window_width / 2)
    upper_bound = window_center + (window_width / 2)
    
    # Clip to window range
    windowed_arr = np.clip(arr, lower_bound, upper_bound)
    
    # Normalize to 0-255
    normalized_arr = ((windowed_arr - lower_bound) / (upper_bound - lower_bound) * 255).astype(np.uint8)
    
    return normalized_arr

def preprocess_pixel_array(dcm):
    """
    Preprocess pixel array with rescaling if needed.
    """
    # Extract raw pixel array
    arr = dcm.pixel_array
    
    # Apply rescale slope and intercept
    if hasattr(dcm, "RescaleSlope") and hasattr(dcm, "RescaleIntercept"):
        slope = dcm.RescaleSlope
        intercept = dcm.RescaleIntercept
        arr = arr * slope + intercept
    
    return arr

def save_dcm_as_png(dcm_file, output_path):
    """
    Read a DICOM file, apply windowing, and save as a PNG using PIL.
    """
    # Read the DICOM file
    dcm = pydicom.dcmread(dcm_file)
    
    # Preprocess pixel array
    pixel_array = preprocess_pixel_array(dcm)
    
    # Get Window Level and Window Width
    window_center = dcm.WindowCenter if hasattr(dcm, "WindowCenter") else 40  # Default values if not present
    window_width = dcm.WindowWidth if hasattr(dcm, "WindowWidth") else 400
    
    # Handle multi-valued WindowCenter or WindowWidth
    if isinstance(window_center, pydicom.multival.MultiValue):
        window_center = window_center[0]
    if isinstance(window_width, pydicom.multival.MultiValue):
        window_width = window_width[0]
    
    # Apply windowing
    windowed_image = apply_windowing(pixel_array, float(window_center), float(window_width))
    
    # Convert to PIL Image
    img = Image.fromarray(windowed_image)
    
    # Save as PNG
    img.save(output_path)


In [36]:
dcm_file_list = glob('../../data/NIA40/CT/**/*.dcm')  # Replace with the path to your DICOM file
output_folder = '../../data/CT/PNG/'  # Replace with the desired output folder
for i in tqdm(range(len(dcm_file_list))):
    file_path=os.path.splitext(os.path.basename(dcm_file_list[i]))[0]
    folder=file_path.split('_')[2]
    create_dir(output_folder+folder+'/'+file_path.split('_')[0]+'_'+file_path.split('_')[1]+'/')
    save_dcm_as_png(dcm_file_list[i], output_folder+folder+'/'+file_path.split('_')[0]+'_'+file_path.split('_')[1]+'/'+file_path+'.png')


100%|██████████| 84890/84890 [56:04<00:00, 25.23it/s]  
